In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler,OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
import pickle

In [2]:
df = pd.read_csv("../data/Churn_Modelling.csv")

In [3]:
df.isna().sum()

RowNumber          0
CustomerId         0
Surname            0
CreditScore        0
Geography          0
Gender             0
Age                0
Tenure             0
Balance            0
NumOfProducts      0
HasCrCard          0
IsActiveMember     0
EstimatedSalary    0
Exited             0
dtype: int64

In [4]:
df.columns

Index(['RowNumber', 'CustomerId', 'Surname', 'CreditScore', 'Geography',
       'Gender', 'Age', 'Tenure', 'Balance', 'NumOfProducts', 'HasCrCard',
       'IsActiveMember', 'EstimatedSalary', 'Exited'],
      dtype='object')

In [5]:
df = df.drop(["RowNumber","CustomerId","Surname"],axis=1)

In [6]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 11 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   CreditScore      10000 non-null  int64  
 1   Geography        10000 non-null  object 
 2   Gender           10000 non-null  object 
 3   Age              10000 non-null  int64  
 4   Tenure           10000 non-null  int64  
 5   Balance          10000 non-null  float64
 6   NumOfProducts    10000 non-null  int64  
 7   HasCrCard        10000 non-null  int64  
 8   IsActiveMember   10000 non-null  int64  
 9   EstimatedSalary  10000 non-null  float64
 10  Exited           10000 non-null  int64  
dtypes: float64(2), int64(7), object(2)
memory usage: 859.5+ KB


In [7]:
X = df.drop("Exited",axis=1)
y = df['Exited']

In [8]:
categorical_columns = X.select_dtypes(include=['object']).columns.tolist()
numerical_columns = X.select_dtypes(include=['int64', 'float64']).columns.tolist()

In [9]:
numerical_columns

['CreditScore',
 'Age',
 'Tenure',
 'Balance',
 'NumOfProducts',
 'HasCrCard',
 'IsActiveMember',
 'EstimatedSalary']

In [10]:
num_pipeline = Pipeline(steps=[
    ('scaler', StandardScaler())
])

cat_pipeline = Pipeline(steps=[
    ('encoder', OneHotEncoder(drop='first', handle_unknown='ignore'))
])


In [11]:
preprocessor = ColumnTransformer(
    transformers=[
        ('num', num_pipeline, numerical_columns),
        ('cat', cat_pipeline, categorical_columns)
    ]
)


In [12]:
X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.2,random_state=123,stratify=y)

In [14]:
X_train_transformed = preprocessor.fit_transform(X_train)
X_test_transformed = preprocessor.transform(X_test)

In [15]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.callbacks import TensorBoard,EarlyStopping
from tensorflow.keras.optimizers import Adam
import datetime

In [16]:
model = Sequential([
    Dense(64, activation='relu', input_shape=(X_train_transformed.shape[1],)),  # Input layer
    Dense(32, activation='relu'),  # Hidden layer
    Dense(1, activation='sigmoid')  # Output layer (for binary classification)
])

In [17]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 64)                768       
                                                                 
 dense_1 (Dense)             (None, 32)                2080      
                                                                 
 dense_2 (Dense)             (None, 1)                 33        
                                                                 
Total params: 2881 (11.25 KB)
Trainable params: 2881 (11.25 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [18]:
# Compile the model
model.compile(
    optimizer=Adam(learning_rate=0.001),
    loss='binary_crossentropy',  # Use 'categorical_crossentropy' for multi-class classification
    metrics=['accuracy']
)

In [19]:
import os
log_dir = os.path.join("logs", "fit", datetime.datetime.now().strftime("%Y%m%d-%H%M%S"))
os.makedirs(log_dir, exist_ok=True)

In [20]:
tensorboard_callback = TensorBoard(log_dir=log_dir, histogram_freq=1)
early_stopping_callback = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)

In [21]:
# Train the model with callbacks
history = model.fit(
    X_train_transformed, y_train,
    epochs=100,
    batch_size=32,
    validation_data=(X_test_transformed,y_test),
    callbacks=[tensorboard_callback, early_stopping_callback],
    verbose=1
)

Epoch 1/100


250/250 [==============================] - 6s 10ms/step - loss: 0.4461 - accuracy: 0.8104 - val_loss: 0.3885 - val_accuracy: 0.8395
Epoch 2/100
250/250 [==============================] - 2s 8ms/step - loss: 0.3762 - accuracy: 0.8447 - val_loss: 0.3524 - val_accuracy: 0.8555
Epoch 3/100
250/250 [==============================] - 2s 10ms/step - loss: 0.3542 - accuracy: 0.8558 - val_loss: 0.3484 - val_accuracy: 0.8570
Epoch 4/100
250/250 [==============================] - 3s 11ms/step - loss: 0.3455 - accuracy: 0.8581 - val_loss: 0.3398 - val_accuracy: 0.8550
Epoch 5/100
250/250 [==============================] - 4s 14ms/step - loss: 0.3400 - accuracy: 0.8594 - val_loss: 0.3451 - val_accuracy: 0.8515
Epoch 6/100
250/250 [==============================] - 3s 13ms/step - loss: 0.3379 - accuracy: 0.8618 - val_loss: 0.3381 - val_accuracy: 0.8520
Epoch 7/100
250/250 [==============================] - 3s 11ms/step - loss: 0.3343 - accuracy: 0.8597 - val_loss: 0.3343 - val_accuracy

In [22]:
# Evaluate the model on the test set
test_loss, test_accuracy = model.evaluate(X_test_transformed, y_test)
print(f'Test Loss: {test_loss:.4f}')
print(f'Test Accuracy: {test_accuracy:.4f}')

63/63 [==============================] - 0s 4ms/step - loss: 0.3326 - accuracy: 0.8600
Test Loss: 0.3326
Test Accuracy: 0.8600


In [23]:
# Make predictions
y_pred = model.predict(X_test_transformed)

# Save the model
model.save('my_ann_model.h5')

63/63 [==============================] - 1s 4ms/step


c:\Users\HW798MH\OneDrive - EY\Desktop\ANN\venv\lib\site-packages\keras\src\engine\training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [24]:
%load_ext tensorboard
%tensorboard --logdir logs/fit


Reusing TensorBoard on port 6006 (pid 33868), started 0:06:05 ago. (Use '!kill 33868' to kill it.)